In [2]:
import torch, os, platform, sys
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

PyTorch: 2.9.0+cu126
CUDA available: True
Fri Nov 28 22:47:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   42C    P8             16W /   72W |       3MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cd /content

!cp /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/mosei_features_and_index.zip .
!unzip -q mosei_features_and_index.zip -d .

In [13]:
# Sync Github repo

!rm -rf /content/HRI-EMO

%cd /content
!git clone https://github.com/Makiato1999/HRI-EMO.git
%cd HRI-EMO

!pwd
!ls

/content
Cloning into 'HRI-EMO'...
remote: Enumerating objects: 363, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 363 (delta 31), reused 42 (delta 18), pack-reused 303 (from 1)
Receiving objects: 100% (363/363), 910.58 KiB | 36.42 MiB/s, done.
Resolving deltas: 100% (207/207), done.
/content/HRI-EMO
/content/HRI-EMO
models	notebooks  README.md  scripts  tests  tools


In [12]:
import torch
from pathlib import Path

def check_modality(name, relative_path):
    print(f"🔎 Checking {name} ...")

    # Assuming current dir is /content/HRI-EMO, so use ../features
    dir_path = Path(relative_path)

    if not dir_path.exists():
        print(f"❌ Path does not exist: {dir_path.resolve()}")
        return False, "PathNotFound"

    try:
        # Get the first .pt file found
        files = list(dir_path.glob("*.pt"))
        if not files:
            print(f"❌ No .pt files found in directory: {dir_path}")
            return False, "NoFiles"

        sample_file = files[0]
        data = torch.load(sample_file, map_location="cpu")
        data_type = type(data)

        if isinstance(data, dict):
            keys = list(data.keys())
            if "hidden" in data:
                print(f"✅ [PASS] Format: Dict | Keys: {keys}")
                return True, "DictWithHidden"
            else:
                print(f"⚠️ [WARNING] Format: Dict | ❌ Missing 'hidden' key | Keys: {keys}")
                return False, "DictNoHidden"

        elif isinstance(data, torch.Tensor):
            print(f"⚠️ [WARNING] Format: Tensor | Shape: {data.shape}")
            return False, "Tensor"

        else:
            print(f"❌ [ERROR] Unknown format: {data_type}")
            return False, "Unknown"

    except Exception as e:
        print(f"❌ Read Error: {e}")
        return False, "Error"

# ================= Run Check =================
print("="*40)
# 1. Check Audio
audio_ok, audio_status = check_modality("Audio", "../features/mosei/seq_level/audio")
print("-" * 20)
# 2. Check Text
text_ok, text_status = check_modality("Text", "../features/mosei/seq_level/text")
print("="*40)

# ================= Final Conclusion =================
print("\n📢 Final Conclusion:")
if audio_ok and text_ok:
    print("🟢 Perfect! Both Audio and Text are standard Dict formats containing 'hidden'.")
    print("🚀 You can run train.py directly. No code changes needed!")
else:
    print("🔴 Formats are inconsistent or non-standard.")
    print("🛠 Please modify the _load_feat function in train.py using the 'Compatible Version' code I provided earlier.")

🔎 Checking Audio ...
✅ [PASS] Format: Dict | Keys: ['hidden', 'attention_mask']
--------------------
🔎 Checking Text ...
✅ [PASS] Format: Dict | Keys: ['hidden', 'attention_mask']

📢 Final Conclusion:
🟢 Perfect! Both Audio and Text are standard Dict formats containing 'hidden'.
🚀 You can run train.py directly. No code changes needed!


In [14]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install tqdm pandas numpy

Looking in indexes: https://download.pytorch.org/whl/cu121


In [15]:
import os, sys

# Check if the repository is in /content/HRI-EMO
print("ls /content ->")
print(os.listdir("/content"))

# If you see the 'HRI-EMO' folder, switch to it
os.chdir("/content/HRI-EMO")
print("✅ cwd:", os.getcwd())

# Ensure the project's 'models' package can be imported
sys.path.append(os.getcwd())

ls /content ->
['.config', 'drive', 'mosei_features_and_index.zip', 'data', 'features', 'HRI-EMO', 'sample_data']
✅ cwd: /content/HRI-EMO


In [ ]:
# ==========================================
# Training / Retraining Section
# ==========================================
%cd /content/HRI-EMO

# ------------------------------------------
# [DEPRECATED] v1 Configuration: Initial Run
# Status: Overfitting observed (High Train AUC, Low Val Performance).
# Issues: Model too complex (2 layers, d_model=384) for MOSEI size.
# ------------------------------------------
# !python -m scripts.fusion.train_mosei_fusion_seq_level_decoder \
#   --index_csv ../data/mosei_index_splits.csv \
#   --audio_dir ../features/mosei/seq_level/audio \
#   --text_dir ../features/mosei/seq_level/text \
#   --epochs 20 \
#   --batch_size 8 \
#   --grad_accum 4 \
#   --warmup_ratio 0.1 \
#   --beta_entropy 1e-3 \
#   --max_len_audio 300 \
#   --max_len_text 128 \
#   --d_model 384 \
#   --n_heads 6 \
#   --num_layers_fusion 2 \
#   --num_layers_decoder 2 \
#   --dropout 0.2 \
#   --lr 1e-4 \
#   --weight_decay 1e-2 \
#   --num_workers 2 \
#   --select_by macro_auc \
#   --save_calibrated_ths \
#   --out_dir /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small \
#   --seed 1234

# ------------------------------------------
# [RECOMMENDED] v2 Configuration: Optimized for Generalization
# Status: Stable. Solved overfitting via strong regularization.
# Key Changes:
#   1. Simplified Architecture: 1 Fusion Layer, d_model=256, beta_hidden=64.
#   2. Stronger Regularization: Dropout=0.4, Weight Decay=0.05.
#   3. Metrics: Checkpointing based on 'calibrated_macro_f1' instead of AUC.
# Note: Uncomment below to run here, or use 'mosei_train.ipynb'.
# ------------------------------------------
!python -m scripts.fusion.train_mosei_fusion_seq_level_decoder \
  --index_csv ../data/mosei_index_splits.csv \
  --audio_dir ../features/mosei/seq_level/audio \
  --text_dir ../features/mosei/seq_level/text \
  --out_dir /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_v2 \
  --epochs 20 \
  --batch_size 16 \
  --grad_accum 2 \
  --lr 5e-5 \
  --weight_decay 0.05 \
  --dropout 0.4 \
  --d_model 256 \
  --n_heads 4 \
  --num_layers_fusion 1 \
  --num_layers_decoder 2 \
  --beta_hidden 64 \
  --beta_entropy 1e-3 \
  --warmup_ratio 0.1 \
  --select_by calibrated_macro_f1 \
  --save_calibrated_ths \
  --seed 1234